In [1]:
import sys
sys.path.insert(0, '../src/libs')

In [7]:
# our file imports
from music_transformer import *
from transformer_training_helpers import *
from song_classification import *
from dataset import *
from midi2audio import FluidSynth

# musicautobot
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.utils.midifile import *
from musicautobot.utils.file_processing import process_all

In [8]:
def topk_decode(model, src, src_mask, max_len, start_symbol, k):
  memory = model.encode(src, src_mask)
  ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
  for i in range(max_len-1):
    out, w2 = model.decode(memory, src_mask,
                       Variable(ys),
                       Variable(subsequent_mask(ys.size(1))
                                .type_as(src.data)))
    prob = model.generator(out[:, -1])
    _, next_words = torch.topk(prob, k = k, dim = 1)
    # if starting, then create 
    #print(next_words)
    if i == 0:
      next_word = next_words[0][0].data.item()
    else:
      k2 = random.randint(0,k-1)
      next_word = next_words[0][k2].data.item()
    #print(next_word)
    #k = random.randint(0,4)
    #next_word = next_words[k].data[0]
    ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
  
  return ys

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out, w2 = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1) # change this for top k
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

Load model from previous state

In [9]:
myModel = torch.load('../models/epochs_30_LM.pt')

UnpicklingError: invalid load key, 'v'.

Load data

In [ ]:
myDs = MidiDataset('../numpy_path', 150, 'N/A', '.npy')
dataloader = DataLoader(myDs, 1, shuffle=True)

for _, item in enumerate(dataloader):
    print(item)
    break

In [ ]:
# data_path = './saved_data/'
# data_save_name = 'music_item_data_0-10000.pkl'
# numpy_path = '../numpy_path'

# numpy_files = get_files(numpy_path, extensions='.npy', recurse=True); len(numpy_files)

# def create_databunch(files, path):
#     #save_file.parent.mkdir(exist_ok=True, parents=True)
#     vocab = MusicVocab.create()
#     processors = [OpenNPFileProcessor(), MusicItemProcessor()]

#     #data = MusicDataBunch.from_files(midi_files, path, processors=processors, encode_position=True)
#     data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)
#     return data

# all_data = create_databunch(numpy_files, data_path); all_data

In [ ]:
print(myDs[0])
new_tens = myDs[0].reshape(myDs[0].shape[0] * myDs[0].shape[1])
print(new_tens.shape)
print(new_tens)

Generate topk/greedy outputs from the model

In [ ]:
myModel.eval()
# ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
#src = Variable(torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]]))
#src = Variable(torch.LongTensor([[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]])).cuda()
idx = 1
src = myDs[idx].reshape(myDs[idx].shape[0] * myDs[idx].shape[1])[:100].unsqueeze(dim=0).long().cuda() # all_data.valid_ds[1][0].to_tensor()[:100].unsqueeze(dim=0)
#print(src.size())
src_mask = Variable(torch.ones(1, 1, 100)).unsqueeze(dim=0).cuda() # 26 was 10
output = greedy_decode(myModel.cuda(), src, src_mask, max_len=100, start_symbol=0)
output2 = topk_decode(myModel.cuda(), src, src_mask, max_len=100, start_symbol=0, k=2)

#print(output2[0])
#print(output[0])

output.cuda()
output2.cuda()

print("GREEDY: ", output[0])
print("TOPK: ", output2[0])
print("SRC: ", src[0])

ncorrect = 0
for i in range(len(src)):
  if src[0][i] == output2[0][i]:
    ncorrect += 1
print("TOPK: ", ncorrect / len(src[0]))

ncorrect = 0
for i in range(len(src)):
  if src[0][i] == output[0][i]:
    ncorrect += 1
print("GREEDY: ", ncorrect / len(src[0]))

Convert outputs to .mid

In [ ]:
%matplotlib inline
from music21 import *
import fluidsynth
#environment.set()
#print(output)
output[0, 1] = 1
output2[0, 1] = 1
#print(src.cpu().numpy())
temp_vocab = MusicVocab.create()
#print(temp_vocab.stoi)
MI = temp_vocab.to_music_item(src.cpu().numpy()[0])
MO = temp_vocab.to_music_item(output.cpu().numpy()[0])
MT = temp_vocab.to_music_item(output2.cpu().numpy()[0])
print("SRC.SHP: ", src.shape)
print("OUTPUT.SHP: ", output.shape)
print("OUTPUT2.SHP: ", output2.shape)
print(src)
print(output)
print(output2)
#print(MI.to_text())
#print(MO.to_text())
#print(MT.to_text())

#new_tens = myDs[0].reshape(myDs[0].shape[0] * myDs[0].shape[1])

MMM = temp_vocab.to_music_item(new_tens.numpy().astype(int))
MI.stream.write('midi', fp='./temp/inp.mid')
MO.stream.write('midi', fp='./temp/pred.mid')
MT.stream.write('midi', fp='./temp/topk.mid')
#MMM.stream.write('midi', fp='./temp_outputs/inp_test.mid')

Classify generated song's musical key

In [ ]:
src_mitem = MusicItem.from_file('./temp/inp.mid', MusicVocab.create())
m_item = MusicItem.from_file('./temp/topk.mid', MusicVocab.create())
m_item2 = MusicItem.from_file('./temp/pred.mid', MusicVocab.create())
print(src_mitem.to_text())
print(int_to_note(34))
print(calculate_score(src_mitem))
print(calculate_score(m_item))
print(calculate_score(m_item2))

Convert .mid to .wav

In [ ]:
filepath = './temp_outputs/inp.mid'
fs = FluidSynth(sound_font='../misc/FluidR3_GM.sf2')
fs.midi_to_audio(filepath, '{}.wav'.format(filepath))

filepath = './temp_outputs/pred.mid'
fs = FluidSynth(sound_font='../misc/FluidR3_GM.sf2')
fs.midi_to_audio(filepath, '{}.wav'.format(filepath))

filepath = './temp_outputs/topk.mid'
fs = FluidSynth(sound_font='../misc/FluidR3_GM.sf2')
fs.midi_to_audio(filepath, '{}.wav'.format(filepath))